**CURSO**: Sensores Remotos, Departamento de Geociencias y Medio Ambiente, Universidad Nacional de Colombia - sede Medellín <br/>
**Profesor**: Edier Aristizábal (evaristizabalg@unal.edu.co) <br/>
**Créditos**: El contenido de este cuaderno proviene de varias fuentes: [CDSE](https://jupyterhub.dataspace.copernicus.eu/user/edieraristizabal@gmail.com/lab/tree/RTC%3Asamples/sentinelhub/from_browser_to_jupyter.ipynb). Se ha hecho todo lo posible por identificar a los titulares de los derechos de autor de los materiales utilizados en este libro. El autor ofrece disculpas por cualquier omisión involuntaria y agradecería incluir el reconocimiento correspondiente en futuras ediciones.

# Request Builder

[Request Builder](https://shapps.dataspace.copernicus.eu/requests-builder/) es una interfaz gráfica en línea para las API de Sentinel Hub. Esta herramienta facilita la creación y depuración de solicitudes a las API, y permite exportar el código resultante en varios lenguajes de programación. En este tutorial, crearemos una solicitud de la [Process API](https://documentation.dataspace.copernicus.eu/APIs/SentinelHub/Process.html) para descargar imágenes ráster del área en Colombia que previamente examinamos en el Browser. Al igual que una [solicitud de Process API en código](https://github.com/eu-cdse/notebook-samples/blob/main/sentinelhub/introduction_to_SH_APIs.ipynb), una solicitud creada con el Request Builder consiste en 5 partes principales:

- `Data Collection`
- `Time Range`
- `Area of Interest`
- `Output`
- `Evalscript`

![True Color Sentinel-2 image of Wildfires near Lac de la Frégate, Quebec, Canada ](https://i.pinimg.com/736x/ed/ea/14/edea14f72e8263558fc1db7f5c3771c1.jpg)

Cada una de estas puede configurarse de manera independiente en la interfaz. Utiliza los siguientes ajustes para el ejemplo :

- `Data Collection`: sentinel-2 l2a
- `Time Range`: Del 01.01.2025 al 30.09.2025
- `Area of Interest`: [-76.048422,5.583184,-74.839341,6.768261]

Haz clic en `Parse` para cargar el área de interés; esta debería mostrarse en la ventana del mapa y acercarse al rectángulo seleccionado.

- `Evalscript`: usa el siguiente evalscript o puedes hacerlo y tomarlo del Copernicus Browser:

```javascript
function setup() {
    return {
        input: ["B02", "B03", "B04", "B08", "B11", "B12", "dataMask"],
        output: [
      		{ id: "default", bands: 4 },
      		{ id: "burnMask", bands: 1, sampleType: "UINT8" },
    	]
      };
}
function evaluatePixel(samples) {
	var NDWI=index(samples.B03, samples.B08); 
	var NDVI=index(samples.B08, samples.B04);
	var INDEX= ((samples.B11 - samples.B12) / (samples.B11 + samples.B12))+(samples.B08);

  	if((INDEX>0.1)||(samples.B02>0.1)||(samples.B11<0.1)||(NDVI>0.3)||(NDWI > 0.1)){
  		return{
            default:[2.5*samples.B04, 2.5*samples.B03, 2.5*samples.B02, samples.dataMask],
			burnMask:[0]
        }
	}
	else {
 	return {
        default: [1, 0, 0, samples.dataMask],
        burnMask: [1]}
	}
}
```

**Output**: el evalscript tiene dos salidas diferentes: default, con 4 bandas (Rojo, Verde, Azul y Transparencia/Data Mask), y burnMask, que devuelve 1 donde el área está quemada y 0 donde no lo está. Debemos configurar la salida para que se cree un ráster a partir de ambas para cada imagen. Como el área de interés es bastante grande y el límite para una sola solicitud es de 2500×2500 píxeles, reduciremos la resolución a 40 metros. Primero, haz clic en la pestaña `resolution`, luego selecciona:

`Res` X in meters: 40

`Res` Y in meters: 40

`Image format`: TIFF

`Identifier`: default

Ahora debes hacer clic en el botón `+ Add` para crear un segundo conjunto de datos de salida. Este debe tener el mismo formato y llevar el nombre correspondiente a la segunda salida del evalscript:

`Image format`: TIFF

`Identifier`: burnMask

Finalmente, debes configurar el lenguaje del `Request Preview` como `python-requests` en el menú desplegable. Esto generará el código de la solicitud que luego podrás usar también en un Jupyter Notebook.

Si ahora haces clic en `Send`, el sistema te pedirá guardar la solicitud y descargar la respuesta. La respuesta será un archivo .tar que contiene default.tif y burnMask.tif.